In [3]:
import sys; sys.path.append('../..')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pyzx as zx
from pyzx.gflow import gflow
from pyzx.heuristics.gflow_calculation import focus_gflow, build_focused_gflow_graph
import math
import copy
from fractions import Fraction
import pickle

In [4]:
def get_broken_matches(gc, gfl):
    broken_matches = set()
    good_match = None
    for match in zx.heuristics.neighbor_unfusion.lcomp_matcher_all(gc, gfl):
        if match[2]: #is neighbor unfusion match
            orig_phase = gc.phase(match[1][0])
            phaseless_s, phase_s = zx.heuristics.neighbor_unfusion.unfuse_to_neighbor(gc, match[1][0], match[2], Fraction(1,2))
            if not zx.gflow.gflow(gc):
    #             print("broken gflow from match ",match[1][0], match[2])
                broken_matches.add(frozenset((match[1][0], match[2])))
            else:
                if not good_match:
                    vn = list(g.neighbors(match[1][0]))
                    boundary = False
                    for neighbor in vn:
                        if g.types()[neighbor] != 1: # no boundaries
                            boundary = True
                    if not boundary:
                        good_match = match

            gc.remove_vertex(phaseless_s)
            gc.remove_vertex(phase_s)
            gc.add_edge(gc.edge(match[1][0], match[2]), 2) #revert all
            gc.set_phase(match[1][0], orig_phase)
    return broken_matches, good_match

def get_extracted_cx_cz_h_stat(g):
    ce = zx.extract_circuit(g.copy())
    ge = ce.to_graph()
    zx.simplify.id_simp(ge)
    c = zx.Circuit.from_graph(ge)
    cx = 0 
    cz = 0
    h = 0
    for g in c.gates:
        if g.name == 'CNOT':
            cx += 1
        if g.name == 'CZ':
            cz += 1
        if g.name == 'HAD':
            h += 1
    return (cx, cz, h)

def calculate_h_edges(g):
    count = 0
    for edge in g.edges():
        if g.edge_type(edge) == 2:
            count += 1
    return count

def edge_list_to_set(edgelist):
    res = []
    for edge in edgelist:
        res.append(frozenset(edge))
    return set(res)

In [5]:
c = zx.Circuit.load('../../evaluation/feyn_bench/before/mod5_4_before').to_basic_gates().split_phase_gates()
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g = zx.simplify.teleport_reduce(g)
g.track_phases = False
g.normalize()
zx.simplify.spider_simp(g)
zx.simplify.to_gh(g)
zx.simplify.id_simp(g)
zx.simplify.spider_simp(g)
zx.draw(g, labels=True, scale=40)

spider_simp: 13. 6. 3. 3. 2. 1.  6 iterations
id_simp: 5.  1 iterations
spider_simp: 4.  1 iterations


In [20]:
# zx.draw(build_focused_gflow_graph(g, focus_gflow(g, gflow(g))), labels=True, scale=40)
get_broken_matches(g, gflow(g)[1])

({frozenset({5, 16}),
  frozenset({5, 13}),
  frozenset({6, 62}),
  frozenset({5, 69}),
  frozenset({7, 28}),
  frozenset({7, 43}),
  frozenset({6, 59}),
  frozenset({27, 28}),
  frozenset({5, 81}),
  frozenset({77, 84}),
  frozenset({7, 40}),
  frozenset({6, 50}),
  frozenset({5, 75}),
  frozenset({39, 46}),
  frozenset({5, 22}),
  frozenset({7, 47}),
  frozenset({5, 78}),
  frozenset({5, 25}),
  frozenset({8, 16}),
  frozenset({6, 66}),
  frozenset({8, 9}),
  frozenset({7, 53}),
  frozenset({7, 34}),
  frozenset({6, 53}),
  frozenset({27, 34}),
  frozenset({58, 65}),
  frozenset({6, 72})},
 (-1.0, (11, [9, 13]), 9))

In [21]:
stat_obj = []
for i in range(0,10):
    gfl = zx.gflow.gflow(g)[1]
    broken, good_match = get_broken_matches(g, gfl)
    stat_obj.append({'g': copy.deepcopy(g), 'broken': broken, 'brokenlen': len(broken), 'h_edges': calculate_h_edges(g), 'ext_gates': get_extracted_cx_cz_h_stat(g), 'apply_match': good_match})
    zx.heuristics.neighbor_unfusion.apply_lcomp(g, good_match, gfl)

id_simp: 44. 4.  2 iterations
apply lcomp match  (-1.0, (11, [9, 13]), 9)
id_simp: 44. 5.  2 iterations
apply lcomp match  (-2.0, (13, [5, 15, 92]), 15)
id_simp: 45. 5.  2 iterations
apply lcomp match  (-1.0, (15, [16, 93]), 16)
id_simp: 46. 5.  2 iterations
apply lcomp match  (-4.0, (16, [8, 5, 21, 95]), 21)
id_simp: 47. 5.  2 iterations
apply lcomp match  (-1.0, (21, [28, 97]), 28)
id_simp: 47. 6.  2 iterations
apply lcomp match  (-2.0, (22, [8, 5, 24]), 8)
id_simp: 48. 6.  2 iterations
apply lcomp match  (0.0, (24, [25, 102, 5]), 25)
id_simp: 49. 6.  2 iterations
apply lcomp match  (-2.0, (25, [5, 27, 103]), 27)
id_simp: 50. 6.  2 iterations
apply lcomp match  (-4.0, (27, [28, 34, 40, 105]), 40)
id_simp: 51. 6.  2 iterations
apply lcomp match  (-3.0, (28, [7, 33, 99, 34, 108, 105]), 33)


In [6]:
with open('stat_obj.pkl', 'rb') as file:
      
    # Call load method to deserialze
    stat_obj = pickle.load(file)

In [7]:
for i in range(0,10):
    zx.draw(stat_obj[i]['g'], labels=True, scale=40)
    zx.draw(build_focused_gflow_graph(stat_obj[i]['g'], focus_gflow(stat_obj[i]['g'], gflow(stat_obj[i]['g']))), labels=True, scale=40)

In [9]:
from pyzx.heuristics.gflow_calculation import *

In [29]:
fg = focus_gflow(stat_obj[9]['g'],gflow(stat_obj[9]['g']))
# get_odd_neighbourhood(stat_obj[9]['g'],fg[1][105])
# stat_obj[9]['g'].vertices()
# stat_obj[9]['g'].graph[107]
fg[1][28]

{33, 39, 49, 52, 58, 68, 71, 74, 77, 87, 88, 89}

In [27]:
i = 0
fggraph = build_focused_gflow_graph(stat_obj[i]['g'], focus_gflow(stat_obj[i]['g'], gflow(stat_obj[i]['g'])))
nc_fggraph = set()
for e in stat_obj[i]['g'].edges():
    if not e in fggraph.edges():
        nc_fggraph.add(frozenset(e))
print(nc_fggraph.symmetric_difference(stat_obj[i]['broken']))
print(len(stat_obj[i]['broken']),len(nc_fggraph))        

set()
27 27


In [28]:
for i in range(0,10):
    fggraph = build_focused_gflow_graph(stat_obj[i]['g'], focus_gflow(stat_obj[i]['g'], gflow(stat_obj[i]['g'])))
    nc_fggraph = set()
    for e in stat_obj[i]['g'].edges():
        if not e in fggraph.edges():
            nc_fggraph.add(frozenset(e))
    print('iteration: ',i)
    print(nc_fggraph.symmetric_difference(stat_obj[i]['broken']))
    print(len(stat_obj[i]['broken']),len(nc_fggraph))
    #         found = False
    #         for cmp_e in stat_obj[0]['broken']:
    #             if set(cmp_e) == set(e):
    #                 found = True
    #         if not found:
    #             print(e)

iteration:  0
set()
27 27
iteration:  1
set()
27 27
iteration:  2
set()
28 28
iteration:  3
set()
28 28
iteration:  4
set()
31 31
iteration:  5
set()
31 31
iteration:  6
set()
32 32
iteration:  7
set()
31 31
iteration:  8
set()
32 32
iteration:  9
{frozenset({105, 108})}
36 35


In [29]:
import pickle
with open('stat_obj.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(stat_obj, file)

In [31]:
with open('stat_obj.pkl', 'rb') as file:
      
    # Call load method to deserialze
    stat2 = pickle.load(file)

In [32]:
stat2

[{'g': Graph(54 vertices, 76 edges),
  'broken': {frozenset({5, 16}),
   frozenset({5, 13}),
   frozenset({6, 62}),
   frozenset({5, 69}),
   frozenset({7, 28}),
   frozenset({7, 43}),
   frozenset({6, 59}),
   frozenset({27, 28}),
   frozenset({77, 84}),
   frozenset({5, 81}),
   frozenset({7, 40}),
   frozenset({6, 50}),
   frozenset({5, 75}),
   frozenset({39, 46}),
   frozenset({5, 22}),
   frozenset({7, 47}),
   frozenset({5, 78}),
   frozenset({5, 25}),
   frozenset({8, 16}),
   frozenset({6, 66}),
   frozenset({8, 9}),
   frozenset({7, 53}),
   frozenset({7, 34}),
   frozenset({6, 53}),
   frozenset({27, 34}),
   frozenset({58, 65}),
   frozenset({6, 72})},
  'brokenlen': 27,
  'h_edges': 67,
  'ext_gates': (0, 27, 40),
  'apply_match': (-1.0, (11, [9, 13]), 9)},
 {'g': Graph(55 vertices, 77 edges),
  'broken': {frozenset({5, 16}),
   frozenset({5, 13}),
   frozenset({6, 62}),
   frozenset({5, 69}),
   frozenset({7, 28}),
   frozenset({7, 43}),
   frozenset({6, 59}),
   frozense

In [13]:
broken2, good_match2 = get_broken_matches(g)
print(len(broken2))

27


In [14]:
broken2

[(15, 5),
 (16, 8),
 (16, 5),
 (22, 5),
 (25, 5),
 (27, 28),
 (27, 34),
 (28, 27),
 (28, 7),
 (34, 27),
 (34, 7),
 (39, 46),
 (40, 7),
 (43, 7),
 (47, 7),
 (50, 6),
 (53, 7),
 (53, 6),
 (58, 65),
 (59, 6),
 (62, 6),
 (66, 6),
 (69, 5),
 (72, 6),
 (75, 5),
 (78, 5),
 (81, 5)]

In [31]:
zx.heuristics.neighbor_unfusion.lcomp_matcher(g, zx.gflow.gflow(g)[1])

[(-2.0, (16, [8, 21, 5, 96]), None),
 (1.0, (21, [16, 109]), None),
 (-2.0, (39, [46, 107, 47]), 46),
 (-2.0, (39, [46, 107, 47]), 107),
 (-2.0, (39, [46, 107, 47]), 47),
 (-2.0, (40, [7, 103, 42]), 7),
 (-2.0, (40, [7, 103, 42]), 103),
 (-2.0, (40, [7, 103, 42]), 42),
 (-1.0, (42, [40, 43]), 40),
 (-1.0, (42, [40, 43]), 43),
 (-2.0, (43, [42, 46, 7]), 42),
 (-2.0, (43, [42, 46, 7]), 46),
 (-2.0, (43, [42, 46, 7]), 7),
 (-2.0, (47, [49, 39, 7]), 49),
 (-2.0, (47, [49, 39, 7]), 39),
 (-2.0, (47, [49, 39, 7]), 7),
 (1.0, (49, [47, 50]), None),
 (-2.0, (50, [49, 6, 52]), 49),
 (-2.0, (50, [49, 6, 52]), 6),
 (-2.0, (50, [49, 6, 52]), 52),
 (-1.0, (52, [50, 53]), 50),
 (-1.0, (52, [50, 53]), 53),
 (-4.0, (53, [52, 7, 6, 58]), 52),
 (-4.0, (53, [52, 7, 6, 58]), 7),
 (-4.0, (53, [52, 7, 6, 58]), 6),
 (-4.0, (53, [52, 7, 6, 58]), 58),
 (-2.0, (58, [65, 53, 66]), 65),
 (-2.0, (58, [65, 53, 66]), 53),
 (-2.0, (58, [65, 53, 66]), 66),
 (-2.0, (59, [6, 7, 61]), 6),
 (-2.0, (59, [6, 7, 61]), 7),
 (

In [33]:
zx.draw(g, labels=True, scale=40)